In [ ]:
from PIL import Image
import numpy
import random
import math

apple = Image.open("banner.png")
array = numpy.array(apple)


for i in range(len(array)):
    for j in range(len(array[i])):
        if array[i][j][0] > 150:
            array[i][j] = [0,min(255,2 * array[i][j][1]),min(255,2 * array[i][j][2]), 255]
        array[i][j][0] += random.uniform(-0.1, 0.1)
        array[i][j][1] += random.uniform(-0.1, 0.1)
        array[i][j][2] += random.uniform(-0.1, 0.1)
        array[i][j][3] += random.uniform(-0.1, 0.1)

apple2 = Image.fromarray(array)
apple2.show()

In [1]:
from PIL import Image
import numpy
import converter
import cv2

leftImg, rightImg = numpy.split(numpy.array(Image.open("img1.png")), 2, axis=1)

def show(img):
    Image.fromarray(img).show()

filteredLeftImg = converter.filter(leftImg)
leftEdgesImage, gLeftIn, rLeftIn = cv2.split(filteredLeftImg)

# show(leftEdgesImage)

filteredRightImg = converter.filter(rightImg)
rightEdgesImage, gRightIn, rRightIn = cv2.split(filteredRightImg)

distances = converter.getDistances(leftEdgesImage, rightEdgesImage)

# image = []

for i in range(len(distances)):
    print(distances[i])
    # image.append([distances[i][2], distances[i][2], distances[i][2]])
# show(image)
# show(numpy.concatenate(([distances], [distances], [distances]), axis=0))
rBlobs, gBlobs = converter.getBlobs(rLeftIn, gLeftIn, rRightIn, gRightIn)

outerWallLandmarks, innerWallLandmarks, rBlobLandmarks, gBlobLandmarks = converter.getLandmarks(distances, rBlobs, gBlobs)

print(outerWallLandmarks, innerWallLandmarks, rBlobLandmarks, gBlobLandmarks)

[] [array([-495.04845887,  503.56983373, -804.32845557,    2.34766153])] [] []


c:\Users\guliz\Documents\GitHub\SPARK_Future-Engineers_2023\Program\Controller\converter.py:98: RuntimeWarning: divide by zero encountered in double_scalars
  dist = wallHeight * focalLength / a[0]
c:\Users\guliz\Documents\GitHub\SPARK_Future-Engineers_2023\Program\Controller\converter.py:99: RuntimeWarning: invalid value encountered in double_scalars
  x = dir * (cameraOffsetX + a[1] * dist)
c:\Users\guliz\Documents\GitHub\SPARK_Future-Engineers_2023\Program\Controller\converter.py:100: RuntimeWarning: invalid value encountered in double_scalars
  y = (cameraOffsetY + a[2] * dist)


In [ ]:
import math

horizontalFov = 155
verticalFov = 115
imageWidth = 272
imageHeight = 154
focalLength = ((imageHeight / 2) / math.tan(math.pi * (verticalFov / 2) / 180))
wallHeight = 10
centerOffset = 10
dist = wallHeight * math.sqrt(focalLength**2 + (imageWidth / 2 - 29)**2) / 36
# return (dir * (3 + a[1] * dist), (10 + a[2] * dist), dist)

print(focalLength)
dist